In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%pip install -U langchain-community chromadb -q

In [3]:
# Main RAG System - Speak with Your Book
# This notebook loads the pre-built model and database to create an interactive chat system

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import chromadb

# 1. CONFIGURATION

In [ ]:
class RAGConfig:
    """Configuration for the RAG System"""

    # Paths
    DATABASE_PATH = "/content/data"
    # Embedding model
    EMBEDDING_MODEL = "Mo7amed3twan/GATE-AraBert-v2"

    LLM_MODEL = "Mo7amed3twan/GradModel-2B-RAG"

    # Model settings
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    MAX_LENGTH = 512
    TEMPERATURE = 0.5

    # Retrieval settings
    RETRIEVAL_K = 2
    CHUNK_SIZE = 1024

In [5]:
config = RAGConfig()

# 2. LOAD MODELS (Embedding & LLM)

In [6]:
def load_arabic_embedding_model():
    try:
        embeddings = HuggingFaceEmbeddings(
            model_name=config.EMBEDDING_MODEL,
            model_kwargs={'device': config.DEVICE},
            encode_kwargs={'normalize_embeddings': True})
        return embeddings

    except Exception as e:
        print(f"Error In Loading The Embedding Model: {e}")
        return None


embeddings = load_arabic_embedding_model()

/tmp/ipython-input-1005417820.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [7]:
def load_arabic_llm():
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            config.LLM_MODEL,
        )

        model = AutoModelForCausalLM.from_pretrained(
            config.LLM_MODEL,
        )

        text_generation_pipeline = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=config.MAX_LENGTH,
            temperature=config.TEMPERATURE,
            do_sample=config.DO_SAMPLE,
            return_full_text=False
        )

        llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
        return llm

    except Exception as e:
        print(f"Error In Loading The LLM : {e}")
        return None

llm = load_arabic_llm()

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-3194935323.py:21: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


# 3. LOAD VECTOR DATABASE

In [ ]:
def load_vector_database():
    try:
        vectorstore = Chroma(
            persist_directory=config.DATABASE_PATH,
            embedding_function=embeddings,
        )

        doc_count = vectorstore._collection.count()
        print(f"{doc_count} Documents in the database")

        return vectorstore

    except Exception as e:
        print(f"Error in loading your database : {e}")
        return None

vectorstore = load_vector_database()

/tmp/ipython-input-4021108195.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


تم تحميل قاعدة البيانات المتجهة تحتوي على 183 مستند


# 4. CREATE RAG CHAIN

In [ ]:
def create_rag_chain():

    arabic_prompt_template = """
أنت مساعد ذكي صممة محمد عطوان لتخدم مشروعه للتخرج (تحدث مع كتابك) تتحدث العربية الفصحي فقط. و مهمتك هي الإجابة على الأسئلة بناءً على السياق المقدم فقط.

السياق:
{context}

السؤال: {question}

التعليمات:
- أجب باللغة العربية الفصحى
- استخدم المعلومات من السياق فقط للإجابة
- إذا لم تجد الإجابة في السياق، قل "لا أعرف" بدلاً من اختلاق إجابة
- كن دقيقاً وواضحاً في إجابتك

الإجابة:
"""

    PROMPT = PromptTemplate(
        template=arabic_prompt_template,
        input_variables=["context", "question"]
    )

    retriever = vectorstore.as_retriever(
        search_kwargs={"k": config.RETRIEVAL_K}
    )

    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True
    )

    return rag_chain

rag_chain = create_rag_chain()

# 5. CHAT INTERFACE

In [13]:
class ArabicChatBot:

    def __init__(self, rag_chain, vectorstore):
        self.rag_chain = rag_chain
        self.vectorstore = vectorstore

    def ask_question(self, question: str):
        if not self.rag_chain:
            return "ُError Loading Rag System (Speck With Your Book)"

        try:
            response = self.rag_chain({"query": question})

            answer = response["result"]
            sources = response["source_documents"]


            return {
                "answer": answer,
                "sources": sources,
                "sources_count": len(sources)
            }

        except Exception as e:
            return f"Error In The Question: {e}"

chatbot = ArabicChatBot(rag_chain, vectorstore)

# 6. TESTING & DEMO

In [ ]:
def main_chat_loop():
    print("\n" + "="*50)
    print("Speck with your Document")
    print("="*50)

    while True:
        try:
            question = input("\nسؤالك (أو 'خروج' للخروج): ").strip()
            
            if question.lower() in ['خروج']:
                print("شكراً لاستخدامك نظام الدردشة")
                break
            if not question:
                continue
            print("جاري المعالجة...")
            
            response = chatbot.ask_question(question)
            
            if isinstance(response, dict):
                print(f"\nالإجابة:")
                print("-" * 20)
                print(response["answer"])
                print(f"\nتم استرجاع {response['sources_count']} جزء ذو صلة")
                
                show_sources = input("\n🔍 عرض الأجزاء المصدر؟ (y/n): ").lower()
                
                if show_sources == 'y':
                    print("\nالأجزاء المصدر:")
                    print("-" * 30)
                    for i, source in enumerate(response["sources"][:3], 1):
                        print(f"\nالجزء {i}:")
                        print(f"المحتوى: {source.page_content[:200]}...")
                        print(f"البيانات الوصفية: {source.metadata}")
            else:
                print(f"{response}")
        
        except KeyboardInterrupt:
            print("\nتم إنهاء الجلسة بواسطة المستخدم")
            break
        
        except Exception as e:
            print(f"خطأ: {e}")

main_chat_loop()


Speck with your Document

سؤالك (أو 'خروج' للخروج): ما هي حقوق الطفل فى القانون المصري؟
جاري المعالجة...

الإجابة:
--------------------
حقوق الطفل في القانون المصري تشمل:

1. الحق في التعليم: يمكن للأطفال أن يحصلوا على التعليم المجاني والمعترف به لمدة 12 عاماً من عمرهم الثامنة عشرة.

2. الحق في الصحة: يجب توفير الرعاية الصحية اللازمة للأطفال، بما في ذلك الرعاية الطبية والعلاجات اللازمة.

3. الحق في الحماية من التعذيب والعنف: يحظر التعذيب بأي شكل من الأشكال، والاعتداء على الأطفال، والتعرض لأي نوع من التجارب الطبية أو العلمية بدون رضا الأطفال.

4. الحق في العيش في بيئته الأم: يجب أن يتمكن الأطفال من العيش في بيئتهم الأم، سواء كانت هذه البيئة تحت الحكم المحلي أو الدولي.

5. الحق في الحياة: يحظر قتل الأطفال عمداً، ويدعو القانون إلى العقوبات المناسبة لقتل الأطفال.

6. الحق في العيش في بيئة آمنة: يجب توفير البيئة الآمنة والأمانة للأطفال، حتى في حالات الصراع أو التوتر.

7. الحق في المشاركة في الحياة السياسية: يمكن للأطفال أن يشاركوا في الحياة السياسية والاجتماعية، ولكن بشكل مناسب ومحترم.

هذ